In [1]:
import numpy as np
import cv2
import os
import glob

In [2]:
# Parameters
board_size = (8, 6)
square_size = 50
num_images = 15
output_dir = ".\\checkboard"

In [3]:
squares_x = board_size[0]+1
squares_y = board_size[1] + 1

In [4]:
# Create directory for images
os.makedirs(output_dir, exist_ok=True)

In [5]:
# STEP 1: Generate perfect chessboard images
for i in range(num_images):
    img = np.ones((squares_y * square_size, squares_x * square_size), np.uint8) * 255
    for y in range(squares_y):
        for x in range(squares_x):
            if (x + y) % 2 == 0:
                cv2.rectangle(
                    img,
                    (x * square_size, y * square_size),
                    ((x + 1) * square_size, (y + 1) * square_size),
                    0,
                    -1
                )
    filename = os.path.join(output_dir, f"calib_{i:02d}.png")
    img_blurred = cv2.GaussianBlur(img, (5, 5), 0)
    cv2.imwrite(filename, img_blurred)

In [6]:
print(f" Generated {num_images} chessboard images in '{output_dir}'.")

 Generated 15 chessboard images in '.\checkboard'.


In [7]:
# STEP 2: Prepare object points
objp = np.zeros((board_size[0] * board_size[1], 3), np.float32)
objp[:, :2] = np.indices(board_size).T.reshape(-1, 2) * square_size

objpoints = []
imgpoints = []

In [8]:
# Termination criteria for corner refinement
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [9]:
# STEP 3: Detect corners
images = glob.glob(os.path.join(output_dir, "*.png"))
collage_images = []

In [10]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, board_size, None)

    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        img_drawn = cv2.drawChessboardCorners(img.copy(), board_size, corners2, ret)
        collage_images.append(img_drawn)

In [11]:
# === Build collage from all images ===
if not collage_images:
    raise ValueError("❌ No corners were detected. Calibration failed.")

In [12]:
# Resize all images to same shape (optional: reduce size for fitting)
collage_images = [cv2.resize(img, (400, 300)) for img in collage_images]

In [13]:
# Arrange images in a grid (e.g., 5 rows x 3 columns)
cols = 5
rows = (len(collage_images) + cols - 1) // cols

In [14]:
# Fill missing slots with white images if needed
white = np.ones_like(collage_images[0]) * 255
while len(collage_images) < rows * cols:
    collage_images.append(white.astype(np.uint8))

In [15]:
# Stack images
rows_img = [np.hstack(collage_images[i * cols:(i + 1) * cols]) for i in range(rows)]
collage = np.vstack(rows_img)

In [16]:
# Display final collage
cv2.imshow("All Detected Chessboards", collage)
cv2.waitKey(0)
cv2.destroyAllWindows()